# NYC Taxi Dataset - Dask for Multi GPU

# Demo 6: A quick intro to Dask + RAPIDS

Dask is a sophisticated package for parallel computation with a number of different datatypes. For much more detail, see: https://tutorial.dask.org/

In these examples, we'll focus on the basics of `dask_cudf` and `dask_cuda`

In [ ]:
import numpy as np
import pandas as pd
import cuml
import cudf
import os

In [ ]:
import dask_cudf

In [ ]:
import dask, dask_cudf
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait

# dask_cudf and dask_cuda are included in the RAPIDS distribution

In [ ]:
# Setup a cluster and connect a client to it
# This will create one "worker" per GPU in your system, allowing you to parallelize tasks over them.

cluster = LocalCUDACluster()
client = Client(cluster)

In [ ]:
client

In [ ]:
%%time

ddf = dask_cudf.read_orc("yellow_tripdata_2014-03-cleaned.orc") # Alternative local approach
# ddf = dask_cudf.read_orc('https://odsc-sample-data.s3-us-west-2.amazonaws.com/yellow_tripdata_2014-03-cleaned.orc')


In [ ]:
len(ddf)

In [ ]:
# Compute a simple histogram of passengers

value_counts = ddf.passenger_count.value_counts()
print(value_counts)

# TODO: ** Now actually show the results here
print(value_counts.compute())

## Machine learning with Dask

We'll show a simple demo of cuML's Random Forest, used in a distributed context. See https://docs.rapids.ai/api/cuml/stable/ for more details on the API, or the blog post (https://medium.com/rapids-ai/accelerating-random-forests-up-to-45x-using-cuml-dfb782a31bea) for implementation details.

In [ ]:
prediction_cols = ["passenger_count", "trip_distance", "rate_code", "fare_amount",
                   "dropoff_latitude", "dropoff_longitude"]

In [ ]:
%%time
X_ddf = ddf[prediction_cols]

# Convert everything to float32
for c in X_ddf.columns:
    X_ddf[c] = X_ddf[c].astype("float32")

Y_ddf = X_ddf["fare_amount"]
X_ddf = X_ddf.drop(columns="fare_amount")

X_ddf, y_ddf = client.persist([X_ddf, Y_ddf]) # Trigger the computation and cache in RAM
_ = wait([X_ddf, y_ddf]) # Actually wait for persistence to finish

In [ ]:
import cuml.dask.ensemble

In [ ]:
model = cuml.dask.ensemble.RandomForestRegressor()
model.fit(X_ddf, y_ddf)

In [ ]:
# TODO: predict (in-sample on training data) and compute an R2 score to make sure we've fit